## Investment Bank Program Type Prediction

Given *data about investment bank program transactions*, let's try to predict the **type** of a given program.

We will use a logistic regression model to make our predictions.

Data source: https://www.kaggle.com/datasets/snassimr/data-for-investing-type-prediction

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('investing_program_prediction_data.csv')
data

,SE1,SE2,BA1,BA2,BA3,BA4,BA5,BA6,BA7,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,PE11,PE12,PE13,PE14,PE15,IA1,IA2,IA3,InvType
0,45,G29,12,0.0,5934,0,0,0,0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,0,0,1,C1
1,41,G0,0,0.0,21514,316374,5142,0,5141,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I1,I1,I1,I0,0,0,1,C1
2,43,G0,1,0.0,14324,22344,22344,0,22344,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,0,0,0,C1
3,36,G52,16,0.0,41114,2406838,1156219,692875,41425,I0,I0,I0,I0,I1,I1,I1,I0,I0,I1,I1,I1,I1,I0,I0,0,0,0,C1
4,2,G10,0,0.0,2394,621,621,0,621,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,0,0,0,C1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4729,57,G23,1,0.0,12514,14274,14274,0,14274,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,0,0,0,C0
4730,34,G66,0,0.0,5994,1036,1036,0,1036,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,0,0,0,C1
4731,44,G51,19,0.0,69627,4805913,3345950,1899126,596537,I0,I0,I0,I1,I1,I0,I1,I0,I0,I1,I1,I1,I1,I1,I0,0,0,1,C1
4732,22,G0,0,0.0,5999,0,0,0,0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,I0,0,0,0,C1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4734 entries, 0 to 4733
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   SE1      4734 non-null   int64  
 1   SE2      4734 non-null   object 
 2   BA1      4734 non-null   int64  
 3   BA2      4734 non-null   float64
 4   BA3      4734 non-null   int64  
 5   BA4      4734 non-null   int64  
 6   BA5      4734 non-null   int64  
 7   BA6      4734 non-null   int64  
 8   BA7      4734 non-null   int64  
 9   PE1      4734 non-null   object 
 10  PE2      4734 non-null   object 
 11  PE3      4734 non-null   object 
 12  PE4      4734 non-null   object 
 13  PE5      4734 non-null   object 
 14  PE6      4734 non-null   object 
 15  PE7      4734 non-null   object 
 16  PE8      4734 non-null   object 
 17  PE9      4734 non-null   object 
 18  PE10     4734 non-null   object 
 19  PE11     4734 non-null   object 
 20  PE12     4734 non-null   object 
 21  PE13     4734 

### Preprocessing

In [4]:
df = data.copy()

In [5]:
df.isna().sum().sum()

0

In [6]:
# Remove non-numeric characters in PE features
for column in ['PE' + str(i) for i in range(1, 16)]:
    df[column] = df[column].apply(lambda x: x[1]).astype(int)

In [7]:
df

,SE1,SE2,BA1,BA2,BA3,BA4,BA5,BA6,BA7,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,PE11,PE12,PE13,PE14,PE15,IA1,IA2,IA3,InvType
0,45,G29,12,0.0,5934,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,C1
1,41,G0,0,0.0,21514,316374,5142,0,5141,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,C1
2,43,G0,1,0.0,14324,22344,22344,0,22344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1
3,36,G52,16,0.0,41114,2406838,1156219,692875,41425,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,0,0,0,C1
4,2,G10,0,0.0,2394,621,621,0,621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4729,57,G23,1,0.0,12514,14274,14274,0,14274,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C0
4730,34,G66,0,0.0,5994,1036,1036,0,1036,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1
4731,44,G51,19,0.0,69627,4805913,3345950,1899126,596537,0,0,0,1,1,0,1,0,0,1,1,1,1,1,0,0,0,1,C1
4732,22,G0,0,0.0,5999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1


In [8]:
# One-hot encode the SE2 column
geo_dummies = pd.get_dummies(df['SE2'], dtype=int)
df = pd.concat([df, geo_dummies], axis=1)
df = df.drop('SE2', axis=1)

In [9]:
df

,SE1,BA1,BA2,BA3,BA4,BA5,BA6,BA7,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,PE11,PE12,PE13,PE14,PE15,IA1,IA2,IA3,InvType,G0,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G2,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,G42,G43,G44,G45,G46,G47,G48,G49,G50,G51,G52,G53,G54,G55,G56,G58,G59,G60,G61,G62,G63,G64,G65,G66,G67,G68,G69,G70,G71,G72,G73,G74,G75,G76,G77,G78,G79,G80,G81,G82,G83,G84,G85,G86,G87,G88,G89,G90,G91,G92,G93,G94,G95,G96,G97,G98,G99
0,45,12,0.0,5934,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41,0,0.0,21514,316374,5142,0,5141,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,C1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,43,1,0.0,14324,22344,22344,0,22344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,36,16,0.0,41114,2406838,1156219,692875,41425,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,0,0,0,C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0.0,2394,621,621,0,621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4729,57,1,0.0,12514,14274,14274,0,14274,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4730,34,0,0.0,5994,1036,1036,0,1036,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4731,44,19,0.0,69627,4805913,3345950,1899126,596537,0,0,0,1,1,0,1,0,0,1,1,1,1,1,0,0,0,1,C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4732,22,0,0.0,5999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# Engineer age features
df['Child'] = df['SE1'].apply(lambda x: 1 if x < 18 else 0)
df['Senior'] = df['SE1'].apply(lambda x: 1 if x >= 65 else 0)

In [11]:
# Engineer activity features
for column in ['BA' + str(i) for i in range(1, 8)]:
    df['Low_' + column] = df[column].apply(lambda x: 1 if x <= df[column].quantile(0.25) else 0)

In [12]:
# Split df into X and y
y = df['InvType']
X = df.drop('InvType', axis=1)

In [13]:
X

,SE1,BA1,BA2,BA3,BA4,BA5,BA6,BA7,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,PE11,PE12,PE13,PE14,PE15,IA1,IA2,IA3,G0,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G2,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,G42,G43,G44,G45,G46,G47,G48,G49,G50,G51,G52,G53,G54,G55,G56,G58,G59,G60,G61,G62,G63,G64,G65,G66,G67,G68,G69,G70,G71,G72,G73,G74,G75,G76,G77,G78,G79,G80,G81,G82,G83,G84,G85,G86,G87,G88,G89,G90,G91,G92,G93,G94,G95,G96,G97,G98,G99,Child,Senior,Low_BA1,Low_BA2,Low_BA3,Low_BA4,Low_BA5,Low_BA6,Low_BA7
0,45,12,0.0,5934,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1
1,41,0,0.0,21514,316374,5142,0,5141,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0
2,43,1,0.0,14324,22344,22344,0,22344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,36,16,0.0,41114,2406838,1156219,692875,41425,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2,0,0.0,2394,621,621,0,621,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4729,57,1,0.0,12514,14274,14274,0,14274,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4730,34,0,0.0,5994,1036,1036,0,1036,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1
4731,44,19,0.0,69627,4805913,3345950,1899126,596537,0,0,0,1,1,0,1,0,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4732,22,0,0.0,5999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1


In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [15]:
X_train

,SE1,BA1,BA2,BA3,BA4,BA5,BA6,BA7,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,PE11,PE12,PE13,PE14,PE15,IA1,IA2,IA3,G0,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G2,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,G42,G43,G44,G45,G46,G47,G48,G49,G50,G51,G52,G53,G54,G55,G56,G58,G59,G60,G61,G62,G63,G64,G65,G66,G67,G68,G69,G70,G71,G72,G73,G74,G75,G76,G77,G78,G79,G80,G81,G82,G83,G84,G85,G86,G87,G88,G89,G90,G91,G92,G93,G94,G95,G96,G97,G98,G99,Child,Senior,Low_BA1,Low_BA2,Low_BA3,Low_BA4,Low_BA5,Low_BA6,Low_BA7
4146,30,1,0.0,3307,521608,521607,0,2374,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1
3152,22,0,0.0,84321,51364,51364,0,51364,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
4227,49,0,0.0,10598,227529,227529,0,227529,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
3287,52,0,0.0,49118,55744,55744,0,55744,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
3444,48,8,0.0,46218,2078840,1263128,902708,289949,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,50,15,0.0,19882,309487,309474,866911,309473,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2763,35,4,0.0,30383,2029542,1022104,0,20526,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
905,57,0,0.0,98904,109100,109100,0,109100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0
3980,23,6,0.0,18598,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1


In [16]:
y_train

4146    C1
3152    C1
4227    C1
3287    C0
3444    C1
        ..
2895    C1
2763    C0
905     C0
3980    C1
235     C0
Name: InvType, Length: 3313, dtype: object

In [17]:
# Scale X with a standard scaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [18]:
X_train

,SE1,BA1,BA2,BA3,BA4,BA5,BA6,BA7,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,PE11,PE12,PE13,PE14,PE15,IA1,IA2,IA3,G0,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G2,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,G42,G43,G44,G45,G46,G47,G48,G49,G50,G51,G52,G53,G54,G55,G56,G58,G59,G60,G61,G62,G63,G64,G65,G66,G67,G68,G69,G70,G71,G72,G73,G74,G75,G76,G77,G78,G79,G80,G81,G82,G83,G84,G85,G86,G87,G88,G89,G90,G91,G92,G93,G94,G95,G96,G97,G98,G99,Child,Senior,Low_BA1,Low_BA2,Low_BA3,Low_BA4,Low_BA5,Low_BA6,Low_BA7
4146,-0.853711,-0.507149,-0.0665,-0.429064,-0.029207,0.343929,-0.281583,-0.391727,-0.235439,-0.057718,-0.087198,-0.077933,1.480444,-0.376333,-0.262782,-0.294255,-0.205309,1.506812,-0.526009,-0.506783,-0.602806,-0.298473,-0.258157,-0.052455,0.0,0.693348,-0.569334,-0.095593,-0.052192,-0.073911,-0.046015,-0.069663,-0.065144,-0.073911,-0.073911,-0.073911,-0.075948,0.0,-0.128723,-0.093972,9.409598,-0.069663,-0.092323,-0.100304,-0.11866,-0.075948,-0.057718,-0.060293,-0.138098,-0.034768,-0.103331,-0.06744,-0.154256,-0.081761,-0.104812,-0.083611,-0.083611,-0.030106,-0.095593,-0.139229,-0.103331,-0.176419,-0.090646,-0.134653,-0.085423,-0.071818,-0.144760,-0.024577,-0.095593,-0.162272,-0.092323,-0.038878,-0.097188,-0.055023,-0.156294,-0.100304,-0.071818,-0.052192,-0.085423,-0.081761,-0.100304,-0.057718,-0.055023,-0.098758,-0.034768,-0.138098,-0.079869,-0.095593,-0.071818,-0.06744,-0.073911,-0.155278,-0.129929,-0.097188,-0.092323,-0.087198,-0.024577,-0.057718,-0.046015,-0.017376,-0.104812,-0.092323,-0.038878,-0.030106,-0.060293,0.0,-0.193861,-0.069663,-0.062765,-0.141465,-0.060293,-0.079869,-0.106274,-0.117345,-0.077933,-0.092323,-0.252118,-0.309142,-0.876824,0.113314,1.743630,-0.571657,-0.571657,0.644366,1.726838
3152,-1.359483,-0.638075,-0.0665,-0.014772,-0.417519,-0.309869,-0.281583,-0.192492,-0.235439,-0.057718,-0.087198,-0.077933,-0.675473,-0.376333,-0.262782,-0.294255,-0.205309,-0.663653,-0.526009,-0.506783,-0.602806,-0.298473,-0.258157,-0.052455,0.0,-0.525899,-0.569334,-0.095593,-0.052192,-0.073911,-0.046015,-0.069663,-0.065144,-0.073911,-0.073911,-0.073911,-0.075948,0.0,-0.128723,-0.093972,-0.106274,-0.069663,-0.092323,-0.100304,-0.11866,-0.075948,-0.057718,-0.060293,-0.138098,-0.034768,-0.103331,-0.06744,-0.154256,-0.081761,-0.104812,-0.083611,-0.083611,-0.030106,-0.095593,-0.139229,-0.103331,-0.176419,-0.090646,-0.134653,-0.085423,-0.071818,-0.144760,-0.024577,-0.095593,-0.162272,-0.092323,-0.038878,-0.097188,-0.055023,-0.156294,-0.100304,-0.071818,-0.052192,-0.085423,-0.081761,-0.100304,-0.057718,-0.055023,-0.098758,-0.034768,-0.138098,-0.079869,10.461039,-0.071818,-0.06744,-0.073911,-0.155278,-0.129929,-0.097188,-0.092323,-0.087198,-0.024577,-0.057718,-0.046015,-0.017376,-0.104812,-0.092323,-0.038878,-0.030106,-0.060293,0.0,-0.193861,-0.069663,-0.062765,-0.141465,-0.060293,-0.079869,-0.106274,-0.117345,-0.077933,-0.092323,-0.252118,-0.309142,1.140480,0.113314,-0.573516,-0.571657,-0.571657,0.644366,-0.579093
4227,0.347499,-0.638075,-0.0665,-0.391779,-0.272048,-0.064940,-0.281583,0.523943,-0.235439,-0.057718,-0.087198,-0.077933,-0.675473,-0.376333,-0.262782,-0.294255,-0.205309,-0.663653,-0.526009,-0.506783,-0.602806,-0.298473,-0.258157,-0.052455,0.0,-0.525899,-0.569334,-0.095593,-0.052192,-0.073911,-0.046015,-0.069663,-0.065144,-0.073911,-0.073911,-0.073911,-0.075948,0.0,-0.128723,-0.093972,-0.106274,-0.069663,-0.092323,-0.100304,-0.11866,-0.075948,-0.057718,-0.060293,-0.138098,-0.034768,-0.103331,-0.06744,-0.154256,-0.081761,-0.104812,-0.083611,-0.083611,-0.030106,-0.095593,-0.139229,-0.103331,-0.176419,-0.090646,-0.134653,-0.085423,-0.071818,-0.144760,-0.024577,-0.095593,-0.162272,-0.092323,-0.038878,-0.097188,-0.055023,-0.156294,-0.100304,-0.071818,-0.052192,-0.085423,-0.081761,-0.100304,-0.057718,-0.055023,-0.098758,-0.034768,-0.138098,-0.079869,-0.095593,-0.071818,-0.06744,-0.073911,-0.155278,-0.129929,-0.097188,-0.092323,-0.087198,-0.024577,-0.057718,-0.046015,-0.017376,-0.104812,-0.092323,-0.0

In [19]:
X_train.mean()

SE1       -1.158144e-16
BA1       -3.646009e-17
BA2        1.179591e-17
BA3        3.431538e-17
BA4        1.501298e-17
               ...     
Low_BA3   -1.554915e-17
Low_BA4   -1.394062e-17
Low_BA5    1.715769e-17
Low_BA6   -2.359182e-17
Low_BA7   -2.091093e-17
Length: 125, dtype: float64

In [20]:
X_train.var()

SE1        1.000302
BA1        1.000302
BA2        1.000302
BA3        1.000302
BA4        1.000302
             ...   
Low_BA3    1.000302
Low_BA4    1.000302
Low_BA5    1.000302
Low_BA6    1.000302
Low_BA7    1.000302
Length: 125, dtype: float64

### Training

In [21]:
model = LogisticRegression()

model.fit(X_train, y_train)

acc = model.score(X_test, y_test)
print("Test Accuracy: {:.2f}%".format(acc*100))

Test Accuracy: 71.15%


With no feature engineering: 67.68% accuracy

With feature engineering (age features): 69.60% accuracy (2% increase)

With feature engineering (age features + activity features): 71.15% accuracy (4% increase)